## Practical 1: word2vec
Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [2]:
import numpy as np
import os
from random import shuffle
import re

In [3]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [4]:
import urllib.request
import zipfile
import lxml.etree

In [5]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [6]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [7]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [8]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [9]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [10]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [11]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [12]:
len(sentences_ted)

266694

In [13]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [14]:
from collections import Counter
counts = Counter()
for sentence in sentences_ted:
    for word in sentence:
        counts[word] += 1
print(counts.most_common(10))
words_top_ted, counts_ted_top1000 = zip(*counts.most_common(1000))

[('the', 207748), ('and', 149305), ('to', 125169), ('of', 114818), ('a', 105399), ('that', 95146), ('i', 83180), ('in', 78070), ('it', 74738), ('you', 70923)]


Plot distribution of top-1000 words

In [15]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [16]:
from gensim.models import Word2Vec

In [17]:
model_ted = Word2Vec(sentences_ted, size=100, window=5, min_count=5, workers=4)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [18]:
model_ted.most_similar("man")

[('woman', 0.8569149971008301),
 ('guy', 0.8231114745140076),
 ('girl', 0.7678636908531189),
 ('lady', 0.7494957447052002),
 ('boy', 0.749032735824585),
 ('soldier', 0.7166836261749268),
 ('gentleman', 0.7146059274673462),
 ('poet', 0.6891442537307739),
 ('kid', 0.6743172407150269),
 ('rabbi', 0.6697818636894226)]

In [19]:
model_ted.most_similar("computer")

[('machine', 0.7381571531295776),
 ('software', 0.7171118259429932),
 ('robot', 0.6859936118125916),
 ('device', 0.6646419763565063),
 ('program', 0.6579735279083252),
 ('video', 0.6308311223983765),
 ('mechanical', 0.6285218000411987),
 ('interface', 0.6250415444374084),
 ('circuit', 0.615089476108551),
 ('camera', 0.6136572360992432)]

In [20]:
from scipy import spatial
print(1-spatial.distance.cosine(model_ted["computer"], model_ted["machine"]))

0.738157133269


#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [21]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [22]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [23]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [24]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [25]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read())

Preprocess sentences (note that it's important to remove small sentences for performance)

In [26]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
#     sentences_wiki[s_i] = sentences_wiki[s_i].split()
del input_text

In [34]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))
print(sentences_wiki)

156099
31219
  n to ensure a victory the germans resorted to unconventional means 


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [39]:
counts = Counter()
for sentence in sentences_wiki:
    for word in sentence:
        counts[word] += 1
print(counts.most_common(10))
words_top_wiki, counts_wiki_top1000 = zip(*counts.most_common(1000))

model_wiki = Word2Vec(sentences_wiki, size=100, window=5, min_count=5, workers=4)

[('the', 51132), ('of', 21966), ('and', 19823), ('in', 17422), ('to', 16028), ('n', 14678), ('a', 14202), ('was', 8430), ('s', 6659), ('on', 5969)]


#### t-SNE visualization

In [44]:
print(model_wiki["b"])

[-0.26602137 -0.12622485 -0.47481298 -0.29492784  0.03533409 -0.2072922
  0.11034894 -0.11347583 -0.08015122 -0.05071733  0.3340537  -0.02163598
  0.06094993 -0.11977492 -0.21032421 -0.27397323  0.11278205 -0.53951025
  0.26672477 -0.08052033  0.18186751  0.13508666  0.14482641 -0.08686136
 -0.35474625  0.01077903 -0.27137306  0.23620988  0.11396401  0.42683986
  0.06080317 -0.21572678  0.25903672 -0.01760312 -0.0267661   0.03326775
  0.01059764 -0.38261792 -0.24591807 -0.3030524   0.02228834  0.21353099
  0.24337147 -0.0662242   0.58028042  0.260548   -0.06003974 -0.19648117
  0.05186296 -0.26287922 -0.13996565 -0.01280112 -0.2632637  -0.32014215
 -0.35008505 -0.17366484  0.04066255 -0.43102515 -0.4323535   0.59151697
  0.00931486  0.45079458  0.25755733 -0.533656   -0.42610031 -0.25383645
 -0.145465    0.1594698  -0.13647656 -0.46764186 -0.30794543  0.04952508
  0.36653873 -0.20343043  0.14675294 -0.33844692  0.1561776  -0.13456871
  0.15755264 -0.08561523 -0.31691152  0.11213743 -0.

In [40]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

KeyError: 'the'

In [ ]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)